In [12]:
import mteb


In [14]:
benchmark = mteb.Benchmark(
    name='MTEB(Medical)',
    tasks=mteb.get_tasks(
        tasks=['Banking77Classification']
        )
)
benchmark



TypeError: Benchmark.__init__() got an unexpected keyword argument 'languages'

In [7]:
benchmark.tasks

MTEBTasks(CUREv1Retrieval(name='CUREv1', languages=['eng', 'fra', 'spa']), NFCorpus(name='NFCorpus', languages=['eng']), TRECCOVID(name='TRECCOVID', languages=['eng']), TRECCOVIDPL(name='TRECCOVID-PL', languages=['pol']), SciFact(name='SciFact', languages=['eng']), SciFactPL(name='SciFact-PL', languages=['pol']), MedicalQARetrieval(name='MedicalQARetrieval', languages=['eng']), PublicHealthQARetrieval(name='PublicHealthQA', languages=['ara', 'eng', 'fra', '...']), MedrxivClusteringP2PFast(name='MedrxivClusteringP2P.v2', languages=['eng']), MedrxivClusteringS2SFast(name='MedrxivClusteringS2S.v2', languages=['eng']), CmedqaRetrieval(name='CmedqaRetrieval', languages=['cmn']), CMedQAv2(name='CMedQAv2-reranking', languages=['cmn']))

In [8]:
from data.get_data import get_single_train_data, SimDataset
from models.utils import add_new_tokens

In [9]:
from transformers import AutoModel, AutoTokenizer

model_path = "answerdotai/ModernBERT-base"
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


path_token_dict = {
    'GleghornLab/abstract_domain_copd': '[COPD]',
    'GleghornLab/abstract_domain_cvd': '[CVD]',
    'GleghornLab/abstract_domain_skincancer': '[CANCER]',
    'GleghornLab/abstract_domain_parasitic': '[PARASITIC]',
    'GleghornLab/abstract_domain_autoimmune': '[AUTOIMMUNE]'
}

token_expert_dict = {
    '[COPD]': 0,
    '[CVD]': 1,
    '[CANCER]': 2,
    '[PARASITIC]': 3,
    '[AUTOIMMUNE]': 4
}


model, tokenizer = add_new_tokens(model, tokenizer, list(path_token_dict.values()))

In [5]:
dataset = get_single_train_data(
    data_path='GleghornLab/abstract_domain_cvd',
    tokenizer=tokenizer,
    path_token_dict=path_token_dict,
    token_expert_dict=token_expert_dict,
    max_length=512,
    add_tokens=True
)


In [ ]:
list(path_token_dict.keys())

In [11]:
import random
import torch
from typing import Any, List, Dict
from torch.utils.data import Dataset as TorchDataset
from datasets import load_dataset


def get_all_train_data(
    data_paths: List[str],
    tokenizer: Any,
    path_token_dict: Dict[str, str],
    token_expert_dict: Dict[str, int],
    max_length: int = 512,
    add_tokens: bool = True,
):
    all_a_documents, all_b_documents, all_expert_assignments = [], [], []
    for path in data_paths:
        domain_token = path_token_dict[path]
        expert_assignment = token_expert_dict[domain_token]
        data = load_dataset(path, split='train').select(range(100))
        all_a_documents.extend(data['a'])
        all_b_documents.extend(data['b'])
        all_expert_assignments.extend([expert_assignment] * len(data['a']))

    random.seed(42)
    entries = list(zip(all_a_documents, all_b_documents, all_expert_assignments))
    random.shuffle(entries)
    all_a_documents, all_b_documents, all_expert_assignments = zip(*entries)
    domain_tokens = list(path_token_dict.values())
    dataset = SimDataset(
        a_documents=all_a_documents,
        b_documents=all_b_documents,
        expert_assignments=all_expert_assignments,
        domain_tokens=domain_tokens,
        tokenizer=tokenizer,
        max_length=max_length,
        add_tokens=add_tokens

    )
    return dataset




In [12]:
dataset = get_all_train_data(
    data_paths=list(path_token_dict.keys()),
    tokenizer=tokenizer,
    path_token_dict=path_token_dict,
    token_expert_dict=token_expert_dict,
    max_length=512,
    add_tokens=True
)

In [13]:
for i in range(100):
    for k, v in dataset[i].items():
        if 'doc' in k:
            print(k)
            print(tokenizer.decode(v['input_ids'][0]))
        else:
            print(v)

a_doc
[CVD]Current approaches to predict cardiovascular risk fail to identify many people who would benefit from preventive treatment, while others receive unnecessary intervention.,Machine-learning offers opportunity to improve accuracy by exploiting complex interactions between risk factors.,We assessed whether machine-learning can improve cardiovascular risk prediction.,Prospective cohort study using routine clinical data of 378,256 patients from UK family practices, free from cardiovascular disease at outset.,Four machine-learning algorithms (random forest, logistic regression, gradient boosting machines, neural networks) were compared to an established algorithm (American College of Cardiology guidelines) to predict first cardiovascular event over 10-years.,Predictive accuracy was assessed by area under the ‘receiver operating curve’ (AUC); and sensitivity, specificity, positive predictive value (PPV), negative predictive value (NPV) to predict 7.5% cardiovascular risk (threshold 